# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
data_file = "Location_cities.csv"
data_file_df = pd.read_csv(data_file)
data_file_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.15,-70.92,35.60,93,40,10.29,CL,1591495903
1,1,Rikitea,-23.12,-134.97,73.65,88,100,30.58,PF,1591496167
2,2,Upernavik,72.79,-56.15,30.42,95,97,12.39,GL,1591496167
3,3,Jamestown,42.10,-79.24,62.60,59,1,12.75,US,1591496139
4,4,Nikolskoye,59.70,30.79,54.00,87,0,4.47,RU,1591496167
...,...,...,...,...,...,...,...,...,...,...
547,547,Labuhan,-6.88,112.21,81.18,79,99,5.12,ID,1591496285
548,548,Margate,51.38,1.39,48.00,76,50,3.00,GB,1591496285
549,549,Manoel Urbano,-8.84,-69.26,74.98,80,0,0.36,BR,1591496285
550,550,Bloomington,44.84,-93.30,64.00,88,90,12.75,US,1591496285


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
hum = data_file_df["Humidity"].astype(float)
max_hum = hum.max()
locations = data_file_df[["Lat", "Lng"]]



In [40]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=hum,dissipating=False, max_intensity=max_hum,point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
hotel_df = data_file_df.loc[(data_file_df["Max Temp"] > 70) & (data_file_df["Max Temp"] < 80) & (data_file_df["Cloudiness"] == 0), :]
hotel_df = hotel_df.dropna(how='any')
hotel_df.reset_index(inplace=True)
del hotel_df['index']
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,8,Faanui,-16.48,-151.75,76.78,58,0,4.18,PF,1591496168
1,55,Tura,25.52,90.22,79.72,86,0,5.19,IN,1591496178
2,110,Mossamedes,-15.20,12.15,71.20,58,0,7.43,AO,1591496190
3,138,Sumbe,-11.21,13.84,72.34,84,0,2.26,AO,1591496197
4,145,Kingaroy,-26.53,151.83,73.00,42,0,4.70,AU,1591496198


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [42]:
hotel_list = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {"location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_list.append(jsn['results'][0]['name'])
    except:
        hotel_.append("")
hotel_df["Hotel Name"] = hotel_list
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,8,Faanui,-16.48,-151.75,76.78,58,0,4.18,PF,1591496168,Vaitape
1,55,Tura,25.52,90.22,79.72,86,0,5.19,IN,1591496178,Tura
2,110,Mossamedes,-15.20,12.15,71.20,58,0,7.43,AO,1591496190,Namibe
3,138,Sumbe,-11.21,13.84,72.34,84,0,2.26,AO,1591496197,Sumbe
4,145,Kingaroy,-26.53,151.83,73.00,42,0,4.70,AU,1591496198,Kingaroy


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [44]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))